<a href="https://colab.research.google.com/github/AlissonRP/LinearReg_to_nn/blob/master/lr_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression from scratch

Here we are going to implement one of the most basic statistical models:
* Linear Regression

We can treat linear regression as a simple case of a neural network (nn) where we have one node and the activation function is the identity
<p align="center"><img align="center" src="https://joshuagoings.com/assets/linear.png" height="370px" width="690"/></p>

## Linear Regression
The linear regression model is simply a structure that predicts the mean ($\mu_i$) of the random variable $Y_i$ using a linear combination of the covariates ($X_i$), that is:

$$
\mu_i = \sum_{i = 0}^{n}w_iX_i
$$

Where $X_i$ is a vector of the column $i$ of the covariate space and $X_0$ = 1.


In practice, we have some observations $(x_i,y_i)$ and we do not know the $w_i$, so some estimation method is necessary. The best known and most used method is the least squares method that minimizes the sum of the squared difference of errors, that is:

$$
\text{min}_{w_i}\sum_i (Y_i - {\mu}_i)²
$$

But here we are going to take an alternative approach, we are going to use the gradient descent as the $w_i$ estimation method

## Gradient descent

The gradient descent method is an optimization method in which the posterior value of the quantity of interest is calculated by taking the current value and going against the gradient direction, as in general we want to minimize a certain function $g_{\theta}$ and the gradient goes in the direction that maximizes $g$.  
Defining $L$ as our loss function, we want to find the values ​​of $w_i$ that minimize $L$, so we have:
$$
w_{i+1} = w_i - \lambda\dfrac{dL}{dw_i}
$$

## Deep Learning and PyTorch

In this section, we will calculate the gradients $\bigg(\dfrac{dL}{dw_i}\bigg)$ using backpropagation and then evaluate the loss, and use the gradient descent to optimize the parameters.

### Why simulated data?

Let's simulate the data of our response variable $y$, by the following function:
$$
y = Xw
$$

Where the $X$ data comes from Poisson distributions with different parameters. We are going to use simulated data because we know **exactly** the value of the parameters, in other words, we know exactly the structure of the data generating function, so we can evaluate the estimation with total precision.

In [31]:
import torch
import numpy as np

In [32]:
np.random.seed(41)

X = torch.from_numpy(np.random.exponential(scale=15,size =(15,4)))
betas = torch.tensor([[1.,3.,15.,-2.]], dtype=torch.float64)
b0 = torch.tensor([[2.]], dtype=torch.float64)
y = X @ betas.t() + b0


So the value of the parameters is specified by the tensor `betas` and `b0`

In [33]:
betas

tensor([[ 1.,  3., 15., -2.]], dtype=torch.float64)

### Hyperparameters

Here we establish some hyperparameters of the nn

* Loss: rmse := $\sqrt{\dfrac{1}{n}\sum_{i= n}^{n}(y - \hat{y})²}$
* Optimizer: Gradient descent := $w_i = w_{i-1} - \lambda \dfrac{d \text{Loss}}{dw_{i-1}}$
* $\lambda$ = 0.001
* Initialization values ​​of the weights come from a standard normal distribution

In [34]:

# loss function
def rmse(y,y_hat):
    return torch.sqrt(((torch.pow(y-y_hat,2))/torch.numel(y)).sum())



In [35]:
w = torch.randn(1,int(X[1].numel()), requires_grad=True, dtype=torch.float64)

b =  torch.randn(1, requires_grad=True, dtype=torch.float64)

b

tensor([-0.9697], dtype=torch.float64, requires_grad=True)

In [36]:
def LinearReg(inputs):
    y_t = inputs @ w.transpose(0,1)  + b
    return y_t
    

In [37]:

loss = rmse(y,LinearReg(X))
loss.backward()
loss

tensor(478.8392, dtype=torch.float64, grad_fn=<SqrtBackward0>)

Here we see that the initial value is very wrong, so we need to optimize this value. The first step is to "clear" the gradients because PyTorch stores them in the object.

In [38]:

w.grad.zero_()
b.grad.zero_()

tensor([0.], dtype=torch.float64)

Here is the step of optimization, as we see this is a loop because we predict the value, calculate the loss and optimize the parameters and make everything again.

In [39]:
learning_rate = 0.001
for i in range(1000):    
    y_hat = LinearReg(X)
    loss = rmse(y,y_hat)
    loss.backward()
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
        w.grad.zero_()
        b.grad.zero_()
    if i == 50 or i==900:
        print(loss)

tensor(415.1686, dtype=torch.float64, grad_fn=<SqrtBackward0>)
tensor(0.8954, dtype=torch.float64, grad_fn=<SqrtBackward0>)


So the parameter values ​​are (except for bias which is in `b`) stored in `w`

In [40]:
w

tensor([[ 1.0540,  3.0817, 15.0093, -1.9342]], dtype=torch.float64,
       requires_grad=True)

In [41]:
y

tensor([[ 261.2693],
        [  60.0724],
        [ 144.7189],
        [ 167.7342],
        [ 329.7891],
        [ 182.9445],
        [ 110.9456],
        [  79.9302],
        [1742.1217],
        [ 450.8701],
        [ 145.6723],
        [ 352.7420],
        [ 117.3411],
        [ 133.8173],
        [ 663.0963]], dtype=torch.float64)

In [42]:
loss

tensor(0.8848, dtype=torch.float64, grad_fn=<SqrtBackward0>)

In [43]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)

In [44]:
from sklearn.metrics import mean_squared_error

mean_squared_error(reg.predict(X), y)

2.562051929831399e-26

In [45]:
reg.predict(X)

array([[ 261.26927572],
       [  60.07240414],
       [ 144.7188992 ],
       [ 167.73416625],
       [ 329.7891077 ],
       [ 182.94448173],
       [ 110.9455657 ],
       [  79.93022758],
       [1742.12174849],
       [ 450.87009971],
       [ 145.67226933],
       [ 352.74195701],
       [ 117.34108074],
       [ 133.81727535],
       [ 663.09627849]])

## Disclaimer
This was done just to learn the basics of PyTorch, evidently the model in this case tends to be overfitting, but the purpose here is not to make predictions but to build the model in a non-traditional approach.